In [ ]:
!pip install -U tfx
!pip install kfp

In [1]:
# Read GCP project id from env.
shell_output=!gcloud config list --format 'value(core.project)' 2>/dev/null
GOOGLE_CLOUD_PROJECT=shell_output[0]
%env GOOGLE_CLOUD_PROJECT={GOOGLE_CLOUD_PROJECT}
print("GCP project ID:" + GOOGLE_CLOUD_PROJECT)

env: GOOGLE_CLOUD_PROJECT=alzheimers-331518
GCP project ID:alzheimers-331518


In [2]:
!gsutil ls gs://{GOOGLE_CLOUD_PROJECT}-kubeflowpipelines-default/tfx-template/data/alzheimers

gs://alzheimers-331518-kubeflowpipelines-default/tfx-template/data/alzheimers/test.tfrecords
gs://alzheimers-331518-kubeflowpipelines-default/tfx-template/data/alzheimers/train.tfrecords
gs://alzheimers-331518-kubeflowpipelines-default/tfx-template/data/alzheimers/Alzheimer_s Dataset/


In [61]:
%%script false --no-raise-error

!pip install kaggle
!mkdir -p ~/.kaggle
!echo '{"username":"dayagan100","key":"6767a772b005bb2f1ac1d7d7cedad2ac"}' > ~/.kaggle/kaggle.json
!chmod 600 ~/.kaggle/kaggle.json

In [15]:
%%script false --no-raise-error

!kaggle datasets download -d tourist55/alzheimers-dataset-4-class-of-images -p ~/data

 38%|██████████████▊                        | 13.0M/34.1M [00:00<00:00, 136MB/s]
100%|███████████████████████████████████████| 34.1M/34.1M [00:00<00:00, 209MB/s]


In [ ]:
%%script false --no-raise-error

!gsutil -m cp -r "/home/jupyter/data/alzheimers/Alzheimer_s Dataset" gs://{GOOGLE_CLOUD_PROJECT}-kubeflowpipelines-default/tfx-template/data/alzheimers/

Copying file:///home/jupyter/data/alzheimers/Alzheimer_s Dataset/train/ModerateDemented/moderateDem22.jpg [Content-Type=image/jpeg]...
Copying file:///home/jupyter/data/alzheimers/Alzheimer_s Dataset/train/ModerateDemented/moderateDem5.jpg [Content-Type=image/jpeg]...
Copying file:///home/jupyter/data/alzheimers/Alzheimer_s Dataset/train/ModerateDemented/moderateDem3.jpg [Content-Type=image/jpeg]...
Copying file:///home/jupyter/data/alzheimers/Alzheimer_s Dataset/train/ModerateDemented/moderateDem40.jpg [Content-Type=image/jpeg]...
Copying file:///home/jupyter/data/alzheimers/Alzheimer_s Dataset/train/ModerateDemented/moderateDem4.jpg [Content-Type=image/jpeg]...
Copying file:///home/jupyter/data/alzheimers/Alzheimer_s Dataset/train/ModerateDemented/moderateDem43.jpg [Content-Type=image/jpeg]...
Copying file:///home/jupyter/data/alzheimers/Alzheimer_s Dataset/train/ModerateDemented/moderateDem25.jpg [Content-Type=image/jpeg]...
Copying file:///home/jupyter/data/alzheimers/Alzheimer_s D

In [21]:
%%script false --no-raise-error

import zipfile 
zf = '/home/jupyter/data/alzheimers-dataset-4-class-of-images.zip'
zip_ref = zipfile.ZipFile(zf, 'r')
zip_ref.extractall('/home/jupyter/data/alzheimers')
zip_ref.close()

In [21]:
%%script false --no-raise-error

!gsutil -m cp -r "/home/jupyter/cmpe297/kubeflow/data/train.tfrecords-000" gs://{GOOGLE_CLOUD_PROJECT}-kubeflowpipelines-default/tfx-template/data/alzheimers/train.tfrecords
!gsutil -m cp -r "/home/jupyter/cmpe297/kubeflow/data/test.tfrecords-000" gs://{GOOGLE_CLOUD_PROJECT}-kubeflowpipelines-default/tfx-template/data/alzheimers/test.tfrecords

Copying file:///home/jupyter/cmpe297/kubeflow/data/train.tfrecords-000 [Content-Type=application/octet-stream]...
- [1/1 files][ 50.5 MiB/ 50.5 MiB] 100% Done                                    
Operation completed over 1 objects/50.5 MiB.                                     
Copying file:///home/jupyter/cmpe297/kubeflow/data/test.tfrecords-000 [Content-Type=application/octet-stream]...
/ [1/1 files][    0.0 B/    0.0 B]                                              
Operation completed over 1 objects.                                              


## Change the below endpoint after creating the Kubeflow pipeline and cluster.

In [37]:
# This refers to the KFP cluster endpoint
ENDPOINT='https://30daed4fdcf7496-dot-us-west1.pipelines.googleusercontent.com' # Enter your ENDPOINT here.
if not ENDPOINT:
    from absl import logging
    logging.error('Set your ENDPOINT in this cell.')

In [38]:
PIPELINE_PATH="~/cmpe297/kubeflow"

%cd {PIPELINE_PATH}

/home/jupyter/cmpe297/kubeflow


In [47]:
!tfx pipeline update \
    --pipeline-path=kubeflow_runner.py \
    --endpoint={ENDPOINT}

CLI
Updating pipeline
Detected Kubeflow.
Use --engine flag if you intend to use a different orchestrator.
[Warning] TFX doesn't depend on skaffold anymore and you can delete the auto-genrated build.yaml file. TFX will NOT build a container even if build.yaml file exists. Use --build-image flag to trigger an image build when creating or updating a pipeline.
INFO:absl:Excluding no splits because exclude_splits is not set.
INFO:absl:Excluding no splits because exclude_splits is not set.
INFO:absl:Excluding no splits because exclude_splits is not set.
INFO:absl:Generating ephemeral wheel package for '/home/jupyter/cmpe297/kubeflow/pipeline/utils_native_keras.py' (including modules: ['pipeline', 'utils_native_keras', 'configs']).
INFO:absl:User module package has hash fingerprint version 76693349a80a169772b7c08e22547e18593c1cec11ba0e8bab9128dd60b9950d.
INFO:absl:Executing: ['/opt/conda/bin/python3.7', '/tmp/tmpo0arv8l5/_tfx_generated_setup.py', 'bdist_wheel', '--bdist-dir', '/tmp/tmpue3kgfk